# Soil Spectral Data Inference for cost-effective monitoring

In [1]:
#!pip install spectres
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
from spectres import spectres

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import time

Loading the soil-spectra.csv <br>
The dataset contains spectra data and measured soil properties <br>
As the first step, we transform the reflectance into absorbance spectra <br>
Second, smoothing data using Savitzky-Golay as digital filter <br>
Third, resample the spectra to 500 to 2450 nm with the spectral resolution of 10 nm <br>
Fourth, Normalize the spectra data using Standard Normal Variate(SNV)

Standard Normal Variate applies two step: <br>
    1. Compute the mean centre of each spectrum $ X_i $ by calculating its mean $ \bar{X_i} $ <br>
    2. Devide the difference of each mean with $ X_i $ by its standard deviation $\sigma_i$, then $X_{snv} = \frac{X_i - \bar{X_i}}{\sigma_i} $

In [35]:
df = pd.read_csv("soil-spectra.csv")
soil = df.iloc[:,2:7]
spectra = df.iloc[:,8:2159]

ab = np.log(100/spectra)  #first step
abs_sg = savgol_filter(ab, window_length = 11, polyorder = 2, deriv = 0)  #second step
wave = np.arange(350, 2501, 1)
new_wave = np.arange(500, 2470, 10)
new_abs = spectres(new_wavs = new_wave, spec_wavs = wave,  spec_fluxes = abs_sg)   #third step

def snv(input_data):
    
    input_data = np.asarray(input_data)
    # Define a new array and populate it with the corrected data  
    output_data = np.zeros_like(input_data)
    for i in range(input_data.shape[0]):
 
        # Apply correction
        output_data[i,:] = (input_data[i,:] - np.mean(input_data[i,:])) / np.std(input_data[i,:])
 
    return output_data

abs_std = snv(new_abs)   #fourth step
abs_std.shape
df

,ID,Depth,pH,Clay (%),Silt (%),Sand (%),SOC (%),N (%),350,351,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,OB2-1-1,0-10,7.34,18,39,43,1.874400,0.190490,18.438100,18.199500,...,46.381433,46.342133,46.287000,46.243700,46.245833,46.272833,46.302433,46.338800,46.377267,46.418400
1,OB2-1-2,20-Oct,7.44,16,42,42,1.333280,0.163019,18.125900,17.772867,...,40.859333,40.843167,40.779400,40.710000,40.675967,40.660267,40.657533,40.666367,40.664833,40.653467
2,OB2-1-3,20-30,7.40,33,30,36,1.041307,0.108395,17.342200,17.121467,...,35.178233,35.174200,35.133100,35.089700,35.069733,35.067067,35.094100,35.143433,35.176300,35.194333
3,OB2-1-4,30-40,7.68,37,18,45,4.936881,0.135670,19.004733,18.644967,...,36.861200,36.783433,36.663800,36.536100,36.431900,36.343233,36.280367,36.238267,36.192600,36.144633
4,OB2-1-5,40-50,7.84,5,28,66,10.405942,0.063240,24.988100,24.574033,...,49.512167,49.308967,49.104567,48.918200,48.778433,48.664767,48.549667,48.439867,48.341467,48.254733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,OB100-3-4,30-40,6.27,23,22,55,0.384374,0.019708,18.679800,18.453400,...,33.916667,33.867167,33.826200,33.784733,33.730833,33.672133,33.614967,33.560567,33.532400,33.530667
193,OB100-3-5,40-50,6.55,20,17,63,0.243897,0.013222,22.507433,22.223467,...,38.621700,38.582633,38.529533,38.461000,38.355300,38.240067,38.169433,38.130200,38.111200,38.114267
194,OB100-3-6,50-60,6.70,21,24,55,0.220232,0.038898,20.264200,20.063800,...,38.961800,38.943633,38.905500,38.851200,38.763500,38.665133,38.600400,38.558833,38.535567,38.532200
195,OB100-3-7,60-70,6.68,30,53,17,0.212419,0.037179,30.855433,30.692567,...,52.624067,52.552333,52.462433,52.373700,52.310867,52.264200,52.231733,52.212733,52.198767,52.190400


Split the data into 70% training and 30% testing

In [3]:
abs_dat = pd.DataFrame(data=abs_std, columns = new_wave)
X = abs_dat
y = soil[['SOC (%)']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size= .3)
#print(X_train)
#print(y_train)



Make a list of models and parameter <br>
Running the code containing both lists to select the best model and hyperparameter setting <br>
Note: .values will give the values in a numpy array (shape: (n,1)) and .ravel will convert that array shape to (n, ) (i.e. flatten it) <br>
    
    ========Please add the code to see the algorithm efficiency i  the for loop (or make another for loop)=========

In [4]:
models = [GradientBoostingRegressor(random_state=0), 
              RandomForestRegressor(), 
                    KNeighborsRegressor()]

params = [{'n_estimators': [10, 50, 100, 150], 'learning_rate': [0.01, 0.25, 1, 1.3], 'max_depth': [1,2]},
             {'n_estimators': [10,50,100,200],'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
                  {'n_neighbors': [1, 2, 3, 4]}]

max_i = -1
max_accuracy = -np.Inf
max_mod = None

bestModelIndex = -1
bestMSE = np.Inf
bestModel = None
times = list()
for i in range(3):
    start = time.time()
    mod = GridSearchCV(models[i], params[i])
    mod.fit(X_train, y_train.values.ravel())
    end = time.time()
    diff = end - start
    times.append(diff)
    test_score = mod.score(X_test, y_test)
    if test_score >= max_accuracy:
        max_accuracy = test_score
        max_i = i
        max_mod = mod.best_estimator_
        
    MSE = mean_squared_error(y_test, mod.predict(X_test))
    if MSE < bestMSE:
        bestModelIndex = i
        bestModel = mod
        bestMSE = MSE
        
print("i:", max_i)
print("Accuracy: ", max_accuracy) 
print("Mean Square Error (MSE): ", bestMSE)
print("Root Mean Square Error (RMSE): ", np.sqrt(bestMSE))
print("The Best Model and Hyperparameter settings:", max_mod)

i: 2
Accuracy:  0.923644085989511
Mean Square Error (MSE):  0.25185498336225154
Root Mean Square Error (RMSE):  0.5018515551059411
The Best Model and Hyperparameter settings: KNeighborsRegressor(n_neighbors=1)


In [5]:
times[max_i] ## time to run best model

for i in range(3):
    print(f"{times[i]} seconds to run {models[i]}")
    


25.177462100982666 seconds to run GradientBoostingRegressor(random_state=0)
86.49457287788391 seconds to run RandomForestRegressor()
0.12706995010375977 seconds to run KNeighborsRegressor()


In [6]:
dp = pd.read_csv("Vis-NIR_Reflectance.csv")
vnir = dp.iloc[:,2:2159]
vnir = vnir.dropna()
print(vnir.head())

ab.p = np.log(100/vnir)  #first step
ab.p_sg = savgol_filter(ab.p, window_length = 11, polyorder = 2, deriv = 0)  #second step
wave = np.arange(350, 2501, 1)
new_wave = np.arange(500, 2470, 10)
new_ab = spectres(new_wavs = new_wave, spec_wavs = wave,  spec_fluxes = ab.p_sg)   #third step

def snv(input_data):
    
    input_data = np.asarray(input_data)
    # Define a new array and populate it with the corrected data  
    output_data = np.zeros_like(input_data)
    for i in range(input_data.shape[0]):
 
        # Apply correction
        output_data[i,:] = (input_data[i,:] - np.mean(input_data[i,:])) / np.std(input_data[i,:])
 
    return output_data

ab.p_std = snv(new_ab)   #fourth step
ab.p_std.shape

/var/folders/dp/g8s606fx4r30xp70s3n58xwr0000gn/T/ipykernel_4552/1181177945.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  dp = pd.read_csv("Vis-NIR_Reflectance.csv")
/var/folders/dp/g8s606fx4r30xp70s3n58xwr0000gn/T/ipykernel_4552/1181177945.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ab.p = np.log(100/vnir)  #first step
/var/folders/dp/g8s606fx4r30xp70s3n58xwr0000gn/T/ipykernel_4552/1181177945.py:7: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ab.p_sg = savgol_filter(ab.p, window_length = 11, polyorder = 2, deriv = 0)  #second step


         350        351        352        353        354        355  \
0  16.124267  15.997933  15.786933  15.537967  15.320267  15.162967   
1  14.367400  14.150533  13.936133  13.721367  13.497767  13.294233   
2  13.215200  13.040767  12.840867  12.630367  12.427567  12.271900   
3  13.919267  13.669767  13.423600  13.181933  12.943767  12.735533   
4  13.143800  13.022733  12.875100  12.706400  12.514367  12.375500   

         356        357        358        359  ...       2491       2492  \
0  15.054000  14.945967  14.893567  14.897767  ...  52.338633  52.315900   
1  13.116833  12.971733  12.890067  12.870000  ...  42.830667  42.765733   
2  12.160500  12.074967  12.038200  12.044933  ...  34.045533  33.988900   
3  12.555600  12.388267  12.275300  12.222833  ...  33.321200  33.278300   
4  12.279367  12.172900  12.090600  12.042967  ...  33.254533  33.208733   

        2493       2494       2495       2496       2497       2498  \
0  52.281267  52.242033  52.203867  52.165267

/var/folders/dp/g8s606fx4r30xp70s3n58xwr0000gn/T/ipykernel_4552/1181177945.py:24: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ab.p_std = snv(new_ab)   #fourth step


(2044, 197)

In [33]:
a = pd.DataFrame(data=ab.p_std, columns = new_wave)
soc_predictions = max_mod.predict(a)
soc_predictions

array([0.87579006, 1.33327985, 0.26707163, ..., 0.17480446, 0.44675002,
       0.17957507])